In [2]:
#Imports and definitions
import sqlite3
import pandas as pd
import os
import pandasql
conn = sqlite3.connect("chinook.db")
def execSQL(conn,query):
  conn.execute(query) # execute an SQL query
  conn.commit() # "commit" that query in order to make its action permanent
def allrowsSelect(conn,query):
  cursor = conn.execute(query)
  for row in cursor:
   print(row)

In [3]:
#Test code
allrowsSelect(conn,"select * from pragma_table_info('invoices')")
#allrowsSelect(conn,"select * from pragma_table_list('invoices')")
#allrowsSelect(conn,"select * from pragma_index_info('invoices')")
#allrowsSelect(conn,"select * from pragma_main_index_xinfo('invoices')")
#allrowsSelect(conn,"select * from pragma_stats")


(0, 'InvoiceId', 'INTEGER', 1, None, 1)
(1, 'CustomerId', 'INTEGER', 1, None, 0)
(2, 'InvoiceDate', 'DATETIME', 1, None, 0)
(3, 'BillingAddress', 'NVARCHAR(70)', 0, None, 0)
(4, 'BillingCity', 'NVARCHAR(40)', 0, None, 0)
(5, 'BillingState', 'NVARCHAR(40)', 0, None, 0)
(6, 'BillingCountry', 'NVARCHAR(40)', 0, None, 0)
(7, 'BillingPostalCode', 'NVARCHAR(10)', 0, None, 0)
(8, 'Total', 'NUMERIC(10,2)', 1, None, 0)


In [4]:
#Problem 1:
#allrowsSelect(conn,"select * from invoices where BillingCountry = 'United Kingdom' order by total desc")
#allrowsSelect(conn,"select BillingCity from invoices where BillingCountry = 'United Kingdom';")
allrowsSelect(conn,"select count(*),BillingCity from invoices where BillingCountry = 'United Kingdom' group by BillingCity;")

#Answer is Edinburgh with 7 invoices

(7, 'Edinburgh ')
(14, 'London')


In [5]:
#Problem 2
execSQL(conn,"drop table if exists invoice_total")
execSQL(conn,'''create table invoice_total as select *
FROM invoice_items
INNER JOIN invoices on invoices.InvoiceId = invoice_items.InvoiceId;''')
allrowsSelect(conn,"select round(avg(c),2) from (select count(TrackId) as c from invoice_total group by CustomerId);")

#37.97 is the average number of tracks purchased


(37.97,)


In [6]:
#Problem 3:
allrowsSelect(conn,'''select count(distinct Title)
FROM albums 
INNER JOIN tracks on albums.AlbumId = tracks.AlbumId where GenreId=2;''')

#13 albums have jazz tracks

(13,)


In [7]:
#Problem 4:
allrowsSelect(conn,'''select count(*),GenreId
FROM tracks 
INNER JOIN invoice_items on tracks.TrackId = invoice_items.TrackId where GenreId=22;''')

#9 comedy tracks were purchased in total

(9, 22)


In [19]:
#Problem 5:
execSQL(conn,"drop table if exists blues_invoices")
allrowsSelect(conn,'''create table blues_invoices as select *
FROM tracks 
INNER JOIN invoice_items on tracks.TrackId = invoice_items.TrackId where GenreId=6;''')
#allrowsSelect(conn,'''select * from blues_invoices;''')
execSQL(conn,"drop table if exists blues_invoices_data")
allrowsSelect(conn,'''create table blues_invoices_data as select *
FROM invoices 
INNER JOIN blues_invoices on blues_invoices.InvoiceId = invoices.InvoiceId;''')
#allrowsSelect(conn,'''select * from blues_invoices_data;''')
execSQL(conn,"drop table if exists blues_invoices_customers")
allrowsSelect(conn,'''create table blues_invoices_customers as select *
FROM customers 
INNER JOIN blues_invoices_data on blues_invoices_data.CustomerId = customers.CustomerId;''')
allrowsSelect(conn,'''select count(*),FirstName,LastName from blues_invoices_customers group by FirstName,LastName order by count(*) desc limit 1;''')
#allrowsSelect(conn,'''select * from blues_invoices_customers;''')

#Leonie Köhler has bought the most blues tracks, at 9 tracks


(9, 'Leonie', 'Köhler')


In [18]:
#Problem 6:
execSQL(conn,"drop table if exists grunge")
execSQL(conn,'''create table grunge as select *
FROM playlist_track
INNER JOIN playlists on playlists.PlaylistId = playlist_track.PlaylistId;''')
#allrowsSelect(conn,'''select * from grunge where PlaylistId=16;''')
execSQL(conn,"drop table if exists grunge_tracks")
execSQL(conn,'''create table grunge_tracks as select *
FROM tracks
INNER JOIN grunge on grunge.TrackId = tracks.TrackId;''')
allrowsSelect(conn,'''select * from grunge_tracks where PlaylistId=16 order by Name limit 2;''')

#Black Hole Sun is the second alphabetically 

(2195, 'Alive', 181, 1, 1, 'Stone Gossard', 341080, 11176623, 0.99, 16, 2195, 16, 'Grunge')
(2516, 'Black Hole Sun', 203, 1, 1, 'Soundgarden', 320365, 10425229, 0.99, 16, 2516, 16, 'Grunge')


In [10]:
#Problem 7:
#allrowsSelect(conn,'''select * from invoice_total;''')
execSQL(conn,"drop table if exists album_track")
execSQL(conn,'''create table album_track as select *
FROM tracks
INNER JOIN albums on albums.AlbumId = tracks.AlbumId;''')
#allrowsSelect(conn,'''select * from album_track;''')
execSQL(conn,"drop table if exists invoice_track")
execSQL(conn,'''create table invoice_track as select *
FROM album_track
INNER JOIN invoice_items on invoice_items.TrackId = album_track.TrackId;''')
allrowsSelect(conn,'''select count(*) from (select sum(UnitPrice) from invoice_track group by AlbumId having sum(UnitPrice)>20);''')

#10 albums have sold more than 20 dollars worth of tracks

(10,)


In [11]:
#Problem 8:
#allrowsSelect(conn,'''select * from invoice_total;''')
execSQL(conn,"drop table if exists supported_by")
execSQL(conn,'''create table supported_by as select *
FROM employees
INNER JOIN customers on customers.SupportRepId = employees.EmployeeId;''')
allrowsSelect(conn,'''select round(avg(c),1) from (select count(*) as c,FirstName,LastName from supported_by group by EmployeeId);''')
#allrowsSelect(conn,'''select FirstName,LastName from supported_by group by EmployeeId;''')
#allrowsSelect(conn,'''select * from customers;''')

#On average 19.7 customers are serviced by each customer support employee.

(19.7,)


In [33]:
#Problem 9:
#allrowsSelect(conn,'''select * from invoice_total;''')
execSQL(conn,"drop table if exists track_genre")
execSQL(conn,'''create table track_genre as select *
FROM genres
INNER JOIN tracks on tracks.GenreId = genres.GenreId;''')
#allrowsSelect(conn,'''select * from album_track;''')
# execSQL(conn,"drop table if exists invoice_track")
# execSQL(conn,'''create table invoice_track as select *
# FROM album_track
# INNER JOIN invoice_total on invoice_total.TrackId = album_track.TrackId;''')
#allrowsSelect(conn,'''select * from track_genre;''')
execSQL(conn,"drop table if exists invoice_genre")
execSQL(conn,'''create table invoice_genre as select *
FROM track_genre
INNER JOIN invoice_items on invoice_items.TrackId = track_genre.TrackId;''')
#allrowsSelect(conn,'''select count(*) from (select sum(UnitPrice) from invoice_genre group by GenreId having sum(UnitPrice)>200);''')
allrowsSelect(conn,'''select count(*) from (select sum(invoice_items.UnitPrice)
 from genres,tracks,invoice_items where genres.GenreId=tracks.GenreId and invoice_items.TrackId=tracks.TrackId
 group by genres.Name having sum(invoice_items.UnitPrice)>200);''')

#4 genres had sales exceeding $200.

(4,)


In [17]:
#Problem 10:
execSQL(conn,"drop table if exists artist_album")
execSQL(conn,'''create table artist_album as select *
FROM albums
INNER JOIN artists on artists.ArtistId = albums.ArtistId;''')
#allrowsSelect(conn,'''select * from artist_album;''')
allrowsSelect(conn,'''select count(*),Name from artist_album group by Name order by count(*) desc limit 3;''')
#allrowsSelect(conn,'''select distinct Name from artist_album order by Name desc;''')

#Deep Purple has the third most albums in the database.

(21, 'Iron Maiden')
(14, 'Led Zeppelin')
(11, 'Deep Purple')


In [16]:
#Problem 11:
execSQL(conn,"drop table if exists track_album")
execSQL(conn,'''create table track_album as select *
FROM albums
INNER JOIN tracks on tracks.AlbumId = albums.AlbumId;''')
allrowsSelect(conn,'''select * from track_album where Title = "Fireball" order by Name limit 3;''')

#Fool is the third track in Fireball in alphabetical order

(60, 'Fireball', 58, 764, "Anyone's Daughter", 60, 1, 1, 'Ritchie Blackmore, Ian Gillan, Roger Glover, Jon Lord, Ian Paice', 284682, 9354480, 0.99)
(60, 'Fireball', 58, 761, 'Fireball', 60, 1, 1, 'Ritchie Blackmore, Ian Gillan, Roger Glover, Jon Lord, Ian Paice', 204721, 6714807, 0.99)
(60, 'Fireball', 58, 766, 'Fools', 60, 1, 1, 'Ritchie Blackmore, Ian Gillan, Roger Glover, Jon Lord, Ian Paice', 500427, 16279366, 0.99)


In [15]:
#Problem 12:
execSQL(conn,"drop table if exists no_artist_album")
execSQL(conn,'''create table no_artist_album as select *
FROM artists
LEFT JOIN albums on artists.ArtistId = albums.ArtistId where AlbumId is NULL;''')
#allrowsSelect(conn, "select * from no_artist_album;")
df = pd.read_sql_query("select count(*) from (select * from no_artist_album);", conn)
#print(df[df['ArtistId'] == None])
print(df)

#71 artists do not have any albums in the database.

   count(*)
0        71


In [36]:
#Problem 13:
#allrowsSelect(conn, "select sum(Milliseconds),Title from track_album group by AlbumId order by sum(Milliseconds);")
pd.read_sql_query("select sum(Milliseconds),Title from track_album group by AlbumId order by sum(Milliseconds);",conn)

#Monteverdi: L'Orfeo is the second shortest album in the database.

,sum(Milliseconds),Title
0,51780,Liszt - 12 Études D'Execution Transcendante
1,66639,Monteverdi: L'Orfeo
2,101293,SCRIABIN: Vers la flamme
3,110266,"Charpentier: Divertissements, Airs & Concerts"
4,120000,Handel: Music for the Royal Fireworks (Origina...
...,...,...
342,59780268,"Heroes, Season 1"
343,63289631,"Lost, Season 2"
344,64854936,"Lost, Season 1"
345,70213784,"Battlestar Galactica (Classic), Season 1"


In [33]:
#Problem 14:
allrowsSelect(conn,"select distinct count(*) from invoices group by BillingCity;")
#allrowsSelect(conn,"select count(*),BillingCity from invoices group by BillingCity;")

#The distinct values for the number of invoices in a city are 6, 7, and 14.

(7,)
(6,)
(14,)
